In [1]:
import sys
  
# setting path
sys.path.append('../counterfactual')

import glob
import os
import fuse, causal_utils
from kl_general import sharpness_correction, torch_mult_fuse
glob.glob("/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/*/")

TE_CONFIG = {
    "N_LABELS": 3,
    "FUSE": torch_mult_fuse,

    "EPOCHS": 16,
    "BATCH_SIZE": 64,
    "LEARNING_RATE": 0.0001
}


In [2]:
!ls /raid/can/nli_models/distill/nli

seed1  seed2  seed3  seed4  seed5


In [3]:
!ls /raid/can/nli_models/baseline/nli/outputs_roberta_large_clark_1_seed13370

ls: cannot access '/raid/can/nli_models/baseline/nli/outputs_roberta_large_clark_1_seed13370': No such file or directory


In [4]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from scipy.special import expit

# select model
# model_path='/raid/can/nli_models/baseline/nli/outputs_roberta_large_clark_1_seed13370/'
model_path='/raid/can/nli_models/distill/nli/seed1/'

# select data path
data_path='/ist/users/canu/debias_nlu/data/nli/'

# select fusion method here
fusion = fuse.sum_fuse

# Get avg  prob of bert

In [5]:
avg=np.array([1/3, 1/3, 1/3])
avg

array([0.33333333, 0.33333333, 0.33333333])

# sharpness correction

In [6]:
df_bias_dev = pd.read_json(
    data_path+'dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
bias_dev_score = [b for b in df_bias_dev['bias_probs']]
bias_dev_score = np.array(bias_dev_score)
# y1m0_dev = fusion(avg, bias_dev_score)
df_bert_dev = pd.read_json('/ist/users/canu/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_dev_matched.jsonl', lines=True)
y1m1prob_dev = []
for p, h in zip(df_bert_dev['probs'], bias_dev_score):
    new_y1m1 = fusion(np.array(p), h)
    y1m1prob_dev.append(new_y1m1)
c = sharpness_correction(bias_dev_score, y1m1prob_dev) 
c = c*np.ones(3)

../counterfactual/kl_general.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  self.target_probs = torch.tensor(target_probs)


In [7]:
c

array([0.11352599, 0.11352599, 0.11352599])

In [8]:
df_bert_dev = pd.read_json('/ist/users/canu/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_dev_matched.jsonl', lines=True)
c2 = sharpness_correction(bias_dev_score, df_bert_dev['probs'],config=TE_CONFIG) 
c2 = c2*np.ones(3)

In [9]:
c2

array([0.54500246, 0.54500246, 0.54500246])

#  get score from the bias model

In [10]:
# avg prob
#s score from bias model
df_hans = pd.read_json(data_path+'hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
# df_hans = pd.read_json(data_path+'test_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
hans_score=[b for b in df_hans['bias_probs'] ]
hans_score=np.array(hans_score)

# y1m0

In [11]:
y1m0prob = fusion(c,hans_score)

In [12]:
result_path=model_path+'normal/'
# bert model predictions on HANS
df_bert = pd.read_json('/ist/users/canu/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_hans.jsonl', lines=True)

In [13]:
# ent = []
y1m1prob = []
for p,h in zip(df_bert['probs'],hans_score):
    new_y1m1 = fusion(np.array(p),h)
    y1m1prob.append(new_y1m1)
  

# corrected y1m0

In [14]:
y1m0prob = fusion(c,hans_score)

# TIE

In [15]:
TIE_A = []
TE_model = []
for p,b in zip(y1m1prob,y1m0prob):
    TIE_A.append(p-b) # TIE  
    
for p,b in zip(df_bert['probs'],hans_score):    
    TE_model.append(p-c2*b)

# Eval HANS

In [16]:
key = {0:"entailment",1:"contradiction",2:"neutral"}
t_labels = []
for i in df_bert['probs']:
    t_labels.append(key[np.argmax(i)])
df_bert['label']=t_labels

labels = []
for i in TIE_A:
    labels.append(key[np.argmax(i)])
df_bert['TIE_debias_label']=labels


labels = []
for i in TE_model:
    labels.append(key[np.argmax(i)])
df_bert['TE_debias_label']=labels


In [17]:
text_ans=""
for idx, obj in enumerate(df_bert['label']):
    text_ans = text_ans + "ex"+str(idx)+","+obj+"\n"   
    
text_ans_debias_tie=""
for idx, obj in enumerate(df_bert['TIE_debias_label']):
    text_ans_debias_tie = text_ans_debias_tie + "ex"+str(idx)+","+obj+"\n"          
    
text_ans_debias_te=""
for idx, obj in enumerate(df_bert['TE_debias_label']):
    text_ans_debias_te = text_ans_debias_te + "ex"+str(idx)+","+obj+"\n"           

In [18]:
def format_label(label):
    if label == "entailment":
        return "entailment"
    else:
        return "non-entailment"

guess_dict = {}
for line in text_ans.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict[parts[0]] = format_label(parts[1])
        
guess_dict_debias_tie = {}
for line in text_ans_debias_tie.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias_tie[parts[0]] = format_label(parts[1])        
        
 
guess_dict_debias_te = {}
for line in text_ans_debias_te.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias_te[parts[0]] = format_label(parts[1])     

In [19]:
labels,baseline_avg=causal_utils.get_heur(guess_dict) # normal

Heuristic entailed results:
lexical_overlap: 0.9998
subsequence: 1.0
constituent: 0.9898

Heuristic non-entailed results:
lexical_overlap: 0.8938
subsequence: 0.3378
constituent: 0.1668
avg: 0.7313333333333335


In [20]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias_tie) # TIE_A

Heuristic entailed results:
lexical_overlap: 0.9994
subsequence: 1.0
constituent: 0.9886

Heuristic non-entailed results:
lexical_overlap: 0.9066
subsequence: 0.3562
constituent: 0.1866
avg: 0.7395666666666667


In [21]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias_te) # TE_model

Heuristic entailed results:
lexical_overlap: 0.9994
subsequence: 1.0
constituent: 0.9858

Heuristic non-entailed results:
lexical_overlap: 0.924
subsequence: 0.3788
constituent: 0.2164
avg: 0.7507333333333334
